In [1]:
import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Lambda, Dense, Dropout, Conv2D, MaxPooling2D, Flatten,Activation,BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.losses import CategoricalCrossentropy

from tensorflow.keras.applications.efficientnet import preprocess_input,EfficientNetB0

from tensorflow.keras import callbacks as cb
from tensorflow.keras import backend as K
from tensorflow.keras import utils
from sklearn.model_selection import train_test_split

gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth( device=gpu, enable=True)

In [2]:
all_classes = glob('/home/jovyan/data/fungi/images/*')
source_classes,target_classes=train_test_split(all_classes,test_size=0.1)
sorce_len=len(source_classes)
target_len=len(target_classes)
print(f"total {len(all_classes)} classes=source {sorce_len} + target {target_len} classes")

total 1394 classes=source 1254 + target 140 classes


In [3]:
W,H,CH = 224,224,3
# W,H,CH = 64,64,3
def load_img(path, width = W):
#     img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB).astype(np.float32)/255.0
    img = tf.io.read_file(path)
#     img = tf.image.decode_jpeg(img, channels=CH)
    img = tf.image.decode_jpeg(img, channels=CH)/255
    img = img.numpy()
    
    shape_dst = np.min(img.shape[:2])
    oh = (img.shape[0] - shape_dst) // 2
    ow = (img.shape[1] - shape_dst) // 2
    center_square = np.array([width,width])// 2
    new_size=(width,width)
    
    # cropping + resize
    img = img[oh:oh + shape_dst, ow:ow + shape_dst]
    img=cv2.resize(img, new_size)
    
    # Random affine+rotation
#     random_state = np.random.RandomState(seed=None)
#     alpha_affine=img.shape[0] * 0.1
#     pts1 = np.float32([center_square + width, 
#                        [center_square[0]+width, center_square[1]-width],
#                        center_square - width])
#     pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
#     M = cv2.getAffineTransform(pts1, pts2)
#     rot_mat = cv2.getRotationMatrix2D(tuple(center_square),random_state.rand()*360., 1.0)
#     img = cv2.warpAffine(img, M, new_size, borderMode=cv2.BORDER_REPLICATE)
#     img = cv2.warpAffine(img, rot_mat, new_size, borderMode=cv2.BORDER_REPLICATE)
    return tf.constant(img)

In [8]:
WAYS = 3
SHOTS=5
QUERIES=1
BATCH_SIZE=4

In [9]:
min([len(glob(sdir+'/*.JPG')) for sdir in source_classes])

6

In [10]:
## exclude classes with too few examples
source_classes=[sdir for sdir in source_classes if len(glob(sdir+'/*.JPG'))>SHOTS+QUERIES]

sorce_len=len(source_classes)
print(f"source {sorce_len} + target {target_len} classes")

source 1242 + target 140 classes


In [11]:
min([len(glob(sdir+'/*.JPG')) for sdir in source_classes])


7

In [12]:
map_fun=lambda string: tf.py_function(func=load_img,inp=[string], Tout=tf.float32)
all_sub = [
    tf.data.Dataset.list_files(sc+'/*.JPG', shuffle=True)
    .map(map_fun)
    for sc in source_classes
]

def gen():
    supports = []
    querys = []
    order=np.random.permutation(len(all_sub))
    for tasks in range(len(all_sub)//WAYS):
        picked=[all_sub[tt] for tt in order[WAYS*tasks:WAYS*(tasks+1)]]
        support = tf.concat(
            [
                next(
                    iter(
                        sub.batch(SHOTS).prefetch(SHOTS)
                    )
                    ) for sub in picked
            ]
            , axis=0)
        idxs=np.random.choice(range(WAYS), size=QUERIES, replace=False)
        query = tf.concat(
            [
                next(
                    iter(
                        picked[idx].batch(1).prefetch(1)
                    )
                    ) for idx in idxs
            ]
            , axis=0)
        yield tf.concat([support, query], axis=0
                       ),tuple(
                          [keras.utils.to_categorical(idx,num_classes=WAYS) for idx in tf.repeat(list(range(WAYS)),SHOTS)]+
                          [keras.utils.to_categorical(idx,num_classes=WAYS) for idx in idxs]
                      )
dd = tf.data.Dataset.from_generator(gen,
                                    output_types=(tf.float32,tuple([tf.float32]*(WAYS*SHOTS+QUERIES))),
                                    output_shapes=((WAYS*SHOTS+QUERIES,W,H,CH),tuple([WAYS]*(WAYS*SHOTS+QUERIES)))
                                   ).batch(BATCH_SIZE)

In [13]:
def pretrain_net(input_shape):
    base_model = EfficientNetB0(weights='imagenet', include_top=False)
    x_in=Input(shape=(W,H,CH))
    x=preprocess_input(x_in)
    xx=Flatten()(base_model(x))
    out=Dense(WAYS)(xx)
    model = Model(inputs=x_in, outputs=out)
    return model

In [14]:
base_dim = (W,H,CH)
base_network = pretrain_net(base_dim)
x_in=Input(shape=(WAYS*SHOTS+QUERIES,W,H,CH))
latent_s=[base_network(x_in[:,ii]) for ii in range(WAYS*SHOTS)]
latent_q=[base_network(x_in[:,WAYS*SHOTS+ii]) for ii in range(QUERIES)]
y=list()
for ss in range(WAYS*SHOTS):
    y.append(tf.nn.softmax(latent_s[ss],axis=-1))
for qq in range(QUERIES):
    y.append(tf.nn.softmax(latent_q[qq],axis=-1))

model = Model(inputs=x_in, outputs=tuple(y))

In [15]:
lr=0.001
def scheduler(epoch):
    global lr
    if epoch % 3 == 0:
        lr /= 2
    return lr
reduce_lr = cb.ReduceLROnPlateau(monitor='loss', factor=0.4,patience=2, min_lr=1e-8)
lr_sched = cb.LearningRateScheduler(scheduler)
tensorboard = cb.TensorBoard()
opt = tf.keras.optimizers.Adam(lr=0.00001)
model.compile(loss=CategoricalCrossentropy(), optimizer=opt, metrics=['categorical_accuracy'])

In [16]:
# %%time
model.fit(dd, epochs=1000, verbose=1,workers=4, callbacks=[reduce_lr,lr_sched, tensorboard])

Epoch 1/1000
      1/Unknown - 0s 329us/step - loss: 26.4087 - tf_op_layer_Softmax_loss: 1.2158 - tf_op_layer_Softmax_1_loss: 2.2094 - tf_op_layer_Softmax_2_loss: 0.7653 - tf_op_layer_Softmax_3_loss: 1.7736 - tf_op_layer_Softmax_4_loss: 1.4960 - tf_op_layer_Softmax_5_loss: 1.5470 - tf_op_layer_Softmax_6_loss: 2.0453 - tf_op_layer_Softmax_7_loss: 1.1895 - tf_op_layer_Softmax_8_loss: 2.8821 - tf_op_layer_Softmax_9_loss: 1.2940 - tf_op_layer_Softmax_10_loss: 1.3373 - tf_op_layer_Softmax_11_loss: 1.6556 - tf_op_layer_Softmax_12_loss: 1.7829 - tf_op_layer_Softmax_13_loss: 1.8355 - tf_op_layer_Softmax_14_loss: 2.1862 - tf_op_layer_Softmax_15_loss: 1.1931 - tf_op_layer_Softmax_categorical_accuracy: 0.2500 - tf_op_layer_Softmax_1_categorical_accuracy: 0.0000e+00 - tf_op_layer_Softmax_2_categorical_accuracy: 0.7500 - tf_op_layer_Softmax_3_categorical_accuracy: 0.2500 - tf_op_layer_Softmax_4_categorical_accuracy: 0.5000 - tf_op_layer_Softmax_5_categorical_accuracy: 0.2500 - tf_op_layer_Softmax_6

KeyboardInterrupt: 